In [1]:
import spacy
import difflib
import pandas as pd
import numpy as np
import jellyfish
import nltk
import sklearn
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from string import punctuation as PUNCTUATIONS
NEGATIONS = ['not','neither','never','no','nobody','none','nor','nothing','nowhere',
             'without','hardly','lack','barely','rarely','seldom'] # exclude negations from stopwords 


pd.options.display.max_colwidth=999
pd.options.display.max_rows=10000

In [10]:
'''
-- * customer_reviews.csv *-----------------------
SELECT reviewid, title, reviewtext, rating, adv_uid
  FROM consumerreview_cdc.REVIEW
  where ModerationStatus = 'APPROVED'
'''

cr_df_all = pd.read_csv('customer_reviews.csv', header=0, low_memory=False)
# cr_df = cr_df_all.iloc[:2000,]
cr_df = cr_df_all
cr_df.head(10)
cr_df.shape

(17263, 5)

In [12]:
nlp = spacy.load('en')
cr_df.head(10)



,REVIEWID,TITLE,REVIEWTEXT,RATING,ADV_UID,Tokens,Text_processed
0,64853844,Outstanding Service,"My wife and I used Optimum First Mortgage to refinance a home loan. We shopped around for the best available rate and luckily found this company. We had the pleasure of working with Tim Williams. It was our first time doing a refinance and Tim did an absolute good job explaining in detail the refinance process, being always available and answering every single question my wife and I had. In all, the whole refinance process was made easy, Tim delivered on his promises and we couId not be more satisfied. Tim and OFM were very professional, prompt and most importantly, honest. I would highly recommend Optimum First Mortgage to anyone.",5,7604,"[My, wife, I, use, Optimum, First, Mortgage, refinance, home, loan, We, shop, available, rate, luckily, company, We, pleasure, work, Tim, Williams, It, ﻿1, time, refinance, Tim, absolute, good, job, explain, refinance, process, available, answer, single, question, wife, I, In, refinance, process, make, easy, Tim, deliver, promise, couId, not, satisfy, Tim, OFM, professional, prompt, importantly, honest, I, highly, recommend, Optimum, First, Mortgage]","My wife I use Optimum First Mortgage refinance home loan . We shop available rate luckily company . We pleasure work Tim Williams . It ﻿1 time refinance Tim absolute good job explain refinance process , available answer single question wife I . In , refinance process make easy , Tim deliver promise couId not satisfy . Tim OFM professional , prompt importantly , honest . I highly recommend Optimum First Mortgage ."
1,64826090,Redundant steps made process way too long,"The refinance seemed to go smoothly. They never indicated any problems, and were always courteous and professional. Any questions I had were answered over the phone or emailed. However, I was asked to sign the same documents online and then on hard copies sent overnight, and then again upon closing. I also received several emails that were computer-generated telling me my refinance was ""conditionally approved"" depending on In spite of their claims that they are self-funded, my loan was held up by requests for further documentation from their underwriters (e.g. two phone calls and an email to verify my employment--already indicated by W-2s and tax returns). In the end, my refinance, which I was told would take about 30 days, took over 100 days.\r\n\r\nI cannot recommend eRates.",1,9139,"[The, refinance, smoothly, They, never, indicate, problem, courteous, professional, Any, question, I, answer, phone, email, However, I, ask, sign, document, online, hard, copy, send, overnight, close, I, receive, email, computer, generate, tell, refinance, conditionally, approve, depend, In, spite, claim, self, fund, loan, hold, request, far, documentation, underwriter, e.g., phone, email, verify, employment, --, indicate, W-2s, tax, return, In, end, refinance, I, tell, 30, day, 100, day, I, not, recommend, eRates]","The refinance smoothly . They never indicate problem , courteous professional . Any question I answer phone email . However , I ask sign document online hard copy send overnight , close . I receive email computer - generate tell refinance "" conditionally approve "" depend In spite claim self - fund , loan hold request far documentation underwriter ( e.g. phone email verify employment -- indicate W-2s tax return ) . In end , refinance , I tell 30 day , 100 day . I not recommend eRates ."
2,64850780,great experience,everything went very smooth and fast,5,9934,"[smooth, fast]",smooth fast
3,64853793,Great experience and best rate,I am glad I contacted Charles for my loan. Best experience ever.,5,2898,"[I, glad, I, contact, Charles, loan, Best, experience]",I glad I contact Charles loan . Best experience .
4,64845204,Very Positive Experience !,I am very happy to work with Dave Ruhlman for refinancing my loan. He helped me to understand all the fees and answered all my other questions. He also

### Noun Phrases

In [14]:
tokens = []
text_ = []
DependencyParDict = {}
 
for doc in nlp.pipe(cr_df.REVIEWTEXT.astype('unicode').values, batch_size=1000, n_threads=-1):

    tokens_ = [tk.lemma_ if tk.lemma_ != '-PRON-' else tk.text.lower() for tk in doc 
    if ((tk.lemma_ not in ENGLISH_STOP_WORDS) or (tk.lemma_ in NEGATIONS)) and tk.lemma_ not in PUNCTUATIONS]
    
    tokens_p = [tk.lemma_ if tk.lemma_ != '-PRON-' else tk.text.lower() for tk in doc 
    if ((tk.lemma_ not in ENGLISH_STOP_WORDS) or (tk.lemma_ in NEGATIONS)) ]
        
    tokens.append(tokens_)
    text_.append(" ".join(tokens_p))
                    
        
cr_df['Tokens'] = tokens
cr_df['Text_processed'] = text_

nouns = set()
for doc in nlp.pipe(cr_df.Text_processed.astype('unicode').values, batch_size=1000, n_threads=-1):
    nouns.update([n.lemma_ for n in doc.noun_chunks])
       

noun_entities_list = list(nouns)
# noun entities of bi_grams
noun_entities_list = [ner for ner in noun_entities_list if len(ner.split())>1 and len(ner.split())<=2 
                      and ('-PRON-' not in ner.split())]
noun_entities_list

['work hour',
 'quick turn',
 'website track',
 'difficult stuff',
 'sebonic step',
 'additional form',
 'term length',
 'brian gillespie',
 'information tracking',
 'colleague work',
 'efficient company',
 'wide year',
 'monthly payment',
 'nick mike',
 'good option',
 'david canas',
 'mcglone nail',
 '0 rating',
 'refinance amerisave',
 'effortless refi',
 'process motion',
 'refinance goal',
 'work deanna',
 'heck difference',
 'transfer document',
 'brock friend',
 'large nw',
 'paul willamette',
 'handle request',
 'entire refi',
 'service representative',
 'senior representative',
 'wendy silvas',
 'bankrate abe',
 'professional caring',
 'good estimation',
 'low dti',
 'realtor friend',
 'dan haney',
 'multiple trip',
 'no automation',
 'app thing',
 'suzanne rachael',
 'jar pruzan',
 'comfortable support',
 'process form',
 'miscommunication appraiser',
 'efficient integra',
 'weekend process',
 'deliver promise',
 'rich velotti',
 'close minute',
 'approval loan',
 'customer s

In [16]:
cr_df.head(100)

,REVIEWID,TITLE,REVIEWTEXT,RATING,ADV_UID,Tokens,Text_processed,Tokens2,Text_processed2
0,64853844,Outstanding Service,"My wife and I used Optimum First Mortgage to refinance a home loan. We shopped around for the best available rate and luckily found this company. We had the pleasure of working with Tim Williams. It was our first time doing a refinance and Tim did an absolute good job explaining in detail the refinance process, being always available and answering every single question my wife and I had. In all, the whole refinance process was made easy, Tim delivered on his promises and we couId not be more satisfied. Tim and OFM were very professional, prompt and most importantly, honest. I would highly recommend Optimum First Mortgage to anyone.",5,7604,"[my, wife, i, use, optimum, mortgage, refinance, home, loan, we, shop, good, available, rate, luckily, company, we, pleasure, work, tim, williams, it, our, time, refinance, tim, absolute, good, job, explain, refinance, process, available, answer, single, question, my, wife, i, refinance, process, make, easy, tim, deliver, his, promise, we, couid, not, satisfied, tim, ofm, professional, prompt, importantly, honest, i, highly, recommend, optimum, mortgage]","my wife i use optimum mortgage refinance home loan . we shop good available rate luckily company . we pleasure work tim williams . it our time refinance tim absolute good job explain refinance process , available answer single question my wife i . , refinance process make easy , tim deliver his promise we couid not satisfied . tim ofm professional , prompt importantly , honest . i highly recommend optimum mortgage .","[my, wife, i, use, optimum, mortgage, refinance, home, loan, we, shop, good, available, rate, luckily, company, we, pleasure, work, tim, williams, it, our, time, refinance, tim, absolute, good, job, explain, refinance, process, available, answer, single, question, my, wife, i, refinance, process, make, easy, tim, deliver, his, promise, we, couid, not, satisfied, tim, ofm, professional, prompt, importantly, honest, i, highly, recommend, optimum, mortgage]","my wife i use optimum mortgage refinance home loan . we shop good available rate luckily company . we pleasure work tim williams . it our time refinance tim absolute good job explain refinance process , available answer single question my wife i . , refinance process make easy , tim deliver his promise we couid not satisfied . tim ofm professional , prompt importantly , honest . i highly recommend optimum mortgage ."
1,64826090,Redundant steps made process way too long,"The refinance seemed to go smoothly. They never indicated any problems, and were always courteous and professional. Any questions I had were answered over the phone or emailed. However, I was asked to sign the same documents online and then on hard copies sent overnight, and then again upon closing. I also received several emails that were computer-generated telling me my refinance was ""conditionally approved"" depending on In spite of their claims that they are self-funded, my loan was held up by requests for further documentation from their underwriters (e.g. two phone calls and an email to verify my employment--already indicated by W-2s and tax returns). In the end, my refinance, which I was told would take about 30 days, took over 100 days.\r\n\r\nI cannot recommend eRates.",1,9139,"[refinance, smoothly, , they, never, indicate, problem, courteous, professional, , question, i, answer, phone, email, , i, ask, sign, document, online, hard, copy, send, overnight, closing, , i, receive, email, computer, generate, tell, me, my, refinance, conditionally, approve, depend, , spite, their, claim, they, self, fund, my, loan, hold, request, documentation, their, underwriter, e.g., phone, email, verify, my, employment, --, indicate, w-2s, tax, return, , end, my, refinance, i, tell, 30, day, 100, day, \r\n\r\n, i, not, recommend, erates]","refinance smoothly . they never indicate problem , courteo

### Collocation

In [17]:
N_TOP_COL = 100 # number of top collocations
words = []
for t in cr_df.Tokens:
    words += t 
    
bgm    = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(words)
scored = finder.score_ngrams( bgm.likelihood_ratio  )
finder.apply_freq_filter(min_freq = 1)
colls = finder.nbest(bgm.likelihood_ratio, N_TOP_COL)

# tri_gm   = nltk.collocations.BigramAssocMeasures()
# finder_tri = nltk.collocations.TrigramCollocationFinder.from_words(words)
# scored_tri = finder_tri.score_ngrams(tri_gm.raw_freq)
# print(scored_tri)

collocation = [" ".join(col) for col in colls]
# collocation


In [18]:
bi_grams = list(set(noun_entities_list).union(set(collocation)))
bi_grams
# len(bi_grams)


['internet refinancing',
 'unclear information',
 'strong review',
 'wide mr',
 'need document',
 'branch person',
 'cash property',
 'portal convenient',
 'work hour',
 'urwashi manage',
 'quick turn',
 'ron trejo',
 'contact step',
 'kyle zotter',
 'difficult stuff',
 'website track',
 'close deal',
 'sebonic step',
 'incredibly ball',
 'mess credit',
 'rickey bowman',
 'additional form',
 'hand deck',
 'capellen watermark',
 'house use',
 'adjustment question',
 'poskitive attitude',
 'sebonic staff',
 'term length',
 'expedient service',
 'man person',
 'greaney cit',
 'single bank',
 'tina make',
 'crapshoot work',
 'brian cassidy',
 'action query',
 'rocket account',
 'use option',
 'expedient manner',
 'flexible option',
 'hamilton person',
 'brian gillespie',
 'lending tree',
 'information tracking',
 'consumerdirect mtg',
 'financing process',
 'colleague work',
 'efficient company',
 'email return',
 'reverse mortgage',
 'couple issue',
 'wide year',
 'final cost',
 'no hide'

### Topic modeling

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(stop_words=None, lowercase=False, max_df=0.9, min_df=0.005)
X = vectorizer.fit_transform(cr_df.Text_processed)
vocabulary_ = vectorizer.get_feature_names()

vocabulary_.extend(bi_grams)
vocabulary_processed = [x for x in vocabulary_ if len(x.split())>=2 or x.isalpha()]
len(vocabulary_processed)

tf_vectorizer = CountVectorizer(stop_words=None, lowercase=False, vocabulary=vocabulary_processed)
tf = tf_vectorizer.fit_transform(cr_df.Text_processed)

tfidf_vectorizer = TfidfVectorizer(stop_words=None, lowercase=False, vocabulary=vocabulary_processed)
tfidf = tfidf_vectorizer.fit_transform(cr_df.Text_processed)


In [126]:
# from sklearn.decomposition import NMF, LatentDirichletAllocation

# no_topics = 20

# # Run NMF
# nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# # Run LDA
# lda = LatentDirichletAllocation(n_components=no_topics, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

# def display_topics(model, feature_names, no_top_words):
#     topic_words = []
#     for topic_idx, topic in enumerate(model.components_):
# #         print ("Topic {}:".format(topic_idx))
#         topic_words.append(" ".join([feature_names[i] 
#                         for i in topic.argsort()[:-no_top_words - 1:-1] if feature_names[i] not in ENGLISH_STOP_WORDS]))
#     return(topic_words)

# no_top_words = 5
# topic_words_nmf = display_topics(nmf, tf_vectorizer.get_feature_names(), no_top_words)
# topic_words_lda = display_topics(lda, tfidf_vectorizer.get_feature_names(), no_top_words)

tagList = ['highly recommend', 'smooth process', 'short quick closing time'
           ,'knowledgeable professional response','great service manner'
           ,'good rate low closing cost fee']




In [127]:

# nlp_vec = spacy.load('en_vectors_web_lg')


SCORE_THRES = 0.8
topic_tag_dict = {}
topic_tag_dict2 = {}
for tidx, topic in enumerate(topic_words_nmf):
    topic_nlp = nlp_vec(topic)
    scores = [t.similarity(topic_nlp) for t in nlp_vec.pipe(tagList)]
    index = np.argmax(scores)
    if scores[index] >= SCORE_THRES:
        topic_tag_dict[str(tidx)]  = tagList[index]
        topic_tag_dict2[(tidx, topic)]  = tagList[index]

# topic_tag_dict
topic_tag_dict2


{(0, 'rate good low closing cost'): 'good rate low closing cost fee',
 (1, 'close time'): 'short quick closing time',
 (9, 'service customer excellent provide'): 'great service manner',
 (10, 'process entire smooth smoothly start'): 'smooth process',
 (14,
  'helpful professional knowledgeable extremely friendly'): 'knowledgeable professional response',
 (19, 'recommend highly mortgage use'): 'highly recommend'}

In [128]:
X = nmf.transform(tfidf)
cr_df['topic_vec'] = pd.Series(list(X))
names = [str(i) for i in range(no_topics)]
cr_df[names] = pd.DataFrame(cr_df.topic_vec.values.tolist(), index= cr_df.index)
cr_df.head(100)

adv_topic_mat = cr_df.groupby("ADV_UID")[names].mean()
for i, col in enumerate(adv_topic_mat.columns):
    if col in topic_tag_dict.keys():
        adv_topic_mat = adv_topic_mat.rename(columns={col:topic_tag_dict[col]}) 
    else:
        adv_topic_mat.drop(col, axis=1, inplace=True)

        
adv_topic_mat/ np.max(adv_topic_mat.values)



,good rate low closing cost fee,short quick closing time,great service manner,smooth process,knowledgeable professional response,highly recommend
ADV_UID,,,,,,
1737.0,0.089861,0.089176,0.081374,0.057040,0.042196,0.064138
1739.0,0.126973,0.091346,0.080829,0.054512,0.062917,0.044450
1748.0,0.167497,0.045481,0.058774,0.016936,0.016121,0.120336
1751.0,0.064846,0.042559,0.073000,0.040202,0.071174,0.041025
1827.0,0.047758,0.075966,0.051762,0.063434,0.108444,0.116945
1861.0,0.108664,0.012938,0.000000,0.101428,0.008436,0.006723
1947.0,0.080781,0.073225,0.046020,0.118295,0.032224,0.045265
1965.0,0.041647,0.052085,0.032911,0.052147,0.023988,0.249390
2007.0,0.104900,0.160647,0.000000,0.098776,0.000000,0.044019


In [132]:
# cr_df.groupby("ADV_UID")["topic_vec"].count()
cr_df.loc[cr_df.ADV_UID==9327.0,]


,ADV_UID,RATING,REVIEWID,REVIEWTEXT,TITLE,Text_processed,Tokens,topic_vec,topic0,topic1,...,10,11,12,13,14,15,16,17,18,19
312,9327.0,1.0,48857985.0,"spend lot of time filling forms but no rates were provided, rather getting junk e-mails now.",Waste of time,"spend lot time form no rate provide , junk e - mail .","[spend, lot, time, form, no, rate, provide, junk, e, mail]","[0.017338145852361726, 0.0011233305533559573, 0.0, 0.0, 0.0005576975434806901, 0.0, 0.0, 0.002448851824415491, 0.00154983252175212, 0.005541503903725364, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.017338,0.001123,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000
527,9327.0,5.0,58359867.0,"I refinanced my loan with All American Lending. The refinance closed in 37 days. My experience isn't that high in refinacning so I don't know the ""art"" of refinancing. I do know that, I was not lied to! I was not taken advantage of! I worked with a company that is reputable, practices responsible lending, they all have great attitudes. I choose them because they were close to my home in Riverside. They checked out on BBB, They have there Lender License, I also learned they are a INC 500 company. Thank you for my great loan! Yes I checked out of loan with others lol and YES All American Lending you gave me a deal!\n\nThank You\n\nAnd you have a gorgeous office!",FANTASTIC! HONEST! TO THE POINT!,"i refinance my loan american lending . refinance close 37 day . my experience not high refinacn i not know "" art "" refinance . i know , i not lie ! i not advantage ! i work company reputable , practice responsible lending , they great attitude . i choose them they close my home riverside . they check bbb , they lender license , i learn they 500 company . thank you my great loan ! yes i check loan lol yes american lending you me deal ! \n\n thank you \n\n you gorgeous office !","[i, refinance, my, loan, american, lending, refinance, close, 37, day, my, experience, not, high, refinacn, i, not, know, art, refinance, i, know, i, not, lie, i, not, advantage, i, work, company, reputable, practice, responsible, lending, they, great, attitude, i, choose, them, they, close, my, home, riverside, they, check, bbb, they, lender, license, i, learn, they, 500, company, thank, you, my, great, loan, yes, i, check, loan, lol, yes, american, lending, you, me, deal, \n\n, thank, you, \n\n, you, gorgeous, office]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.01378006986699178, 0.042070464504358855, 0.025367507673141602, 0.03883969400393048, 0.0, 0.0, 0.0, 0.0, 0.019308691711106832, 0.0, 0.0, 0.0, 0.0, 0.005719464701978759, 2.967487711103007e-05]",0.000000,0.000000,...,0.000000,0.000000,0.000000,0.019309,0.000000,0.000000,0.0,0.0,0.005719,0.00003
4753,9327.0,5.0,48890033.0,I just finished my conventional home mortgage refinance with All American Lending everything they told me was dead on. They closed my loan a few days before originally quoted me. Very happy with there service! Everything that was quoted was exactly what I got!,All American Lending Honest & Ethical!,i just finish my conventional home mortgage refinance american lending they tell me dead . they close my loan day originally quote me . happy service ! quote exactly i !,"[i, just, finish, my, conventional, home, mortgage, refinance, american, lending, they, tell, me, dead, they, close, my, loan, day, originally, quote, me, happy, service, quote, exactly, i]","[0.0012509722285407436, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034544899747628505, 0.025665558523892948, 0.014213597569334902, 0.0, 0.0, 0.0, 0.01262194448723748, 0.0, 0.0001757906059572546, 0.0, 0.0, 0.0, 0.0]",0.001251,0.000000,...,0.000000,0.000000,0.000000,0.012622,0.000000,0.000176,0.0,0.0,0.000000,0.00000
13047,9327.0,5.0,49865424.0,My loan officer Tiffany was so knowledgable and achieved my goal of wanting a lower monthly payment! I am so happy with my services here,Super knowledgable!,my loan officer tiffany knowledgable achieve my goal want low monthly p

### Dependency parsing

In [162]:
nlp = spacy.load('en')
aa= nlp(u"there is hasn't countries that deny human basic civil rights")
[n.lemma_.strip() for n in aa if n.lemma_.strip() not in PUNCTUATIONS]


['there',
 'be',
 'have',
 'not',
 'country',
 'that',
 'deny',
 'human',
 'basic',
 'civil',
 'right']

### Synonyms

In [225]:
import spacy

from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

# Load an spacy model (supported models are "es" and "en") 
nlp = spacy.load('en')
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')
token1 = nlp('smooth closing process')
token2 = nlp('rate good low closing cost')
token2.similarity(token1)
# wordnet object link spacy token with nltk wordnet interface by giving acces to
# synsets and lemmas 
# aa = token._.wordnet.synsets()
# token._.wordnet.lemmas()

# And automatically tags with wordnet domains
# token._.wordnet.wordnet_domains()
# aa = token._.wordnet.wordnet_synsets_for_domain(['home','finance'])
# # Imagine we want to enrich the following sentence with synonyms
# sentence = nlp('I want to withdraw 5,000 euros')

# # spaCy WordNet lets you find synonyms by domain of interest
# # for example economy
# economy_domains = ['finance', 'banking']
# enriched_sentence = []

# # For each token in the sentence
# for token in sentence:
#     # We get those synsets within the desired domains
#     synsets = token._.wordnet.wordnet_synsets_for_domain(economy_domains)
#     if synsets:
#         lemmas_for_synset = []
#         for s in synsets:
#             # If we found a synset in the economy domains
#             # we get the variants and add them to the enriched sentence
#             lemmas_for_synset.extend(s.lemma_names())
#             enriched_sentence.append('({})'.format('|'.join(set(lemmas_for_synset))))
#     else:
#         enriched_sentence.append(token.text)

# # Let's see our enriched sentence
# print(' '.join(enriched_sentence))
# # >> I (need|want|require) to (draw|withdraw|draw_off|take_out) 5,000 euros

0.8179546474280619